In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
from pydataset import data
import matplotlib.pyplot as plt
import acquire
import prepare
from scipy import stats
import os
from env import get_connection
import warnings
warnings.filterwarnings("ignore")

# 1. Attendance Data

Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

You should end up with something like this:


name<br>
Billy    0.5250<br>
Jane     0.6875<br>
John     0.9125<br>
Sally    0.7625<br>
Name: grade, dtype: float64

In [28]:
untidy_df = pd.read_sql('SELECT * FROM attendance', get_connection('tidy_data'))
untidy_df

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [29]:
df = untidy_df.rename(columns={'Unnamed: 0':'name'})
df

,name,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [30]:
df = df.melt(id_vars='name', var_name='date', value_name='attend')
df

,name,date,attend
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T
5,Jane,2018-01-02,P
6,Billy,2018-01-02,T
7,John,2018-01-02,T
8,Sally,2018-01-03,T
9,Jane,2018-01-03,T


Notes: 
<br>P = Present
<br>H = half-day
<br>A = Absent
<br>T = Tardy

In [33]:
df.attend = df.attend.map({'P':1, 'H':.5, 'A':0, 'T':.9})
df

,name,date,attend
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0
3,John,2018-01-01,1.0
4,Sally,2018-01-02,0.9
5,Jane,2018-01-02,1.0
6,Billy,2018-01-02,0.9
7,John,2018-01-02,0.9
8,Sally,2018-01-03,0.9
9,Jane,2018-01-03,0.9


In [34]:
df.groupby('name').mean()

,attend
name,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


# 2. Coffee Levels

a. Read the coffee_levels table.<br>
b. Transform the data so that each carafe is in it's own column.
<br> c. Is this the best shape for the data?

In [35]:
untidy_df = pd.read_sql('SELECT * FROM coffee_levels', get_connection('tidy_data'))
untidy_df

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


In [36]:
untidy_df.columns.to_list()

['hour', 'coffee_carafe', 'coffee_amount']

In [37]:
df = untidy_df.rename(columns={'coffee_carafe':'carafe',
                               'coffee_amount': 'amount'})
df

,hour,carafe,amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


In [38]:
df.pivot_table(index='hour', columns='carafe')


amount                    
carafe         x         y         z
hour                                
8       0.816164  0.189297  0.999264
9       0.451018  0.521502  0.915990
10      0.843279  0.023163  0.144928
11      0.335533  0.235529  0.311495
12      0.898291  0.017009  0.771947
13      0.310711  0.997464  0.398520
14      0.507288  0.058361  0.864464
15      0.215043  0.144644  0.436364
16      0.183891  0.544676  0.280621
17      0.391560  0.594126  0.436677

**This is not the best shape to display the data**

# 3. Cake Recipes

a. Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures. <br>
b. Tidy the data as necessary.<br>
c. Which recipe, on average, is the best? Recipe B <br>
d. Which oven temperature, on average, produces the best results? 275<br>
e. Which combination of recipe, rack position, and temperature gives the best result? Recipe R, bottom rack, 300 degrees with a score of 99.25<br>

In [43]:
untidy_df = pd.read_sql('SELECT * FROM cake_recipes', get_connection('tidy_data'))
untidy_df

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


In [52]:
df = untidy_df.melt(id_vars='recipe:position', var_name='temp', value_name='score')
df

,recipe:position,temp,score
0,a:bottom,225,61.738655
1,a:top,225,51.709751
2,b:bottom,225,57.095320
3,b:top,225,82.455004
4,c:bottom,225,96.470207
5,c:top,225,71.306308
6,d:bottom,225,52.799753
7,d:top,225,96.873178
8,a:bottom,250,53.912627
9,a:top,250,52.009735


In [53]:
df[['recipe','position']] = df['recipe:position'].str.split(':', expand=True)
df

,recipe:position,temp,score,recipe,position
0,a:bottom,225,61.738655,a,bottom
1,a:top,225,51.709751,a,top
2,b:bottom,225,57.095320,b,bottom
3,b:top,225,82.455004,b,top
4,c:bottom,225,96.470207,c,bottom
5,c:top,225,71.306308,c,top
6,d:bottom,225,52.799753,d,bottom
7,d:top,225,96.873178,d,top
8,a:bottom,250,53.912627,a,bottom
9,a:top,250,52.009735,a,top


In [81]:
df = df.drop(columns = 'recipe:position')
df

,temp,score,recipe,position
0,225,61.738655,a,bottom
1,225,51.709751,a,top
2,225,57.095320,b,bottom
3,225,82.455004,b,top
4,225,96.470207,c,bottom
5,225,71.306308,c,top
6,225,52.799753,d,bottom
7,225,96.873178,d,top
8,250,53.912627,a,bottom
9,250,52.009735,a,top


In [82]:
df.groupby('recipe').mean().idxmax()

score    b
dtype: object

In [83]:
df.groupby('recipe').mean().max()

score    76.736074
dtype: float64

In [84]:
df.groupby('temp').mean().sort_values('score').idxmax()

score    275
dtype: object

In [89]:
df.groupby(['recipe','position','temp']).mean().sort_values('score')

,,,score
recipe,position,temp,
a,top,300,50.228470
d,top,300,50.971626
a,top,225,51.709751
d,bottom,275,51.747686
c,bottom,250,52.001358
a,top,250,52.009735
d,bottom,225,52.799753
a,bottom,250,53.912627
c,top,300,53.960273


In [79]:
df.groupby(['recipe', 'position', 'temp']).mean().sort_values('score').tail(1)

,,,score
recipe,position,temp,
b,bottom,300,99.248541
